In [1]:
import csv
import math
import string
import itertools
from io import open
from conllu import parse_incr
import torch
import torch.nn as nn
import torch.nn.functional as F
from allennlp.commands.elmo import ElmoEmbedder
elmo = ElmoEmbedder()

Better speed can be achieved with apex installed from https://www.github.com/nvidia/apex.


In [2]:
from torch.nn import CosineEmbeddingLoss
import torch.optim as optim

sense_vec = torch.ones(3, 1)
sense_vec.requires_grad = True
def_vec = torch.randn(3, 1)
def_vec.requires_grad = True

opt = optim.Adam([sense_vec, def_vec])
l = CosineEmbeddingLoss()
test_vec = torch.ones(def_vec.size())

for i in range(1):
    
    opt.zero_grad()
    loss = l(sense_vec, def_vec, test_vec)
    print(loss)
    loss.backward()
    print(sense_vec.grad)
    print(def_vec.grad)
    
    opt.step()

print(sense_vec)

tensor(0., grad_fn=<MeanBackward1>)
tensor([[0.],
        [0.],
        [0.]])
tensor([[0.0000e+00],
        [1.1921e-07],
        [0.0000e+00]])
tensor([[1.],
        [1.],
        [1.]], requires_grad=True)


In [3]:
'''
return: 
all sentences
target word index
target word lemma
target word sense
all senses for each word 
from the EUD for train, test, and dev dataset
index provided by WSD dataset by White et. al.
'''
# get all the senses and definitions for each word from WSD dataset
# order of senses and definitions are in order
def get_all_senses_and_definitions(wsd_data):

    # all senses for each 
    all_senses = {}
    all_definitions = {}
    
    # for test purpose: only load specific amount of data
    for i in range(60):

        # get the original sentence from EUD
        sentence_id = wsd_data[i].get('Sentence.ID')
        
        # remove punctuations from definitions?
        definition = wsd_data[i].get('Sense.Definition').translate(str.maketrans('', '', string.punctuation)).split(' ')
        
        # the index in EUD is 1-based!!!
        sentence_number = int(sentence_id.split(' ')[-1]) - 1
        word_index = int(wsd_data[i].get('Arg.Token')) - 1
        word_lemma = wsd_data[i].get('Arg.Lemma')
        word_sense = wsd_data[i].get('Synset')
        response = wsd_data[i].get('Sense.Response')

        # if the word already exits: add the new sense to the list
        # else: creata a new list for the word
        if word_lemma in all_senses.keys():
            if word_sense not in all_senses[word_lemma]:
                all_senses[word_lemma].append(word_sense)
        else:
            all_senses[word_lemma] = []
            all_senses[word_lemma].append(word_sense)            
            
        if word_lemma in all_definitions.keys():
            if definition not in all_definitions[word_lemma]:
                # split to list before store
                all_definitions[word_lemma].append(definition)
        else:
            all_definitions[word_lemma] = []
            all_definitions[word_lemma].append(definition)
        
    return all_senses, all_definitions

In [4]:
# parse the WSD dataset first
# and retrieve all sentences from the EUD

'''
Copyright@
White, A. S., D. Reisinger, K. Sakaguchi, T. Vieira, S. Zhang, R. Rudinger, K. Rawlins, & B. Van Durme. 2016. 
[Universal decompositional semantics on universal dependencies]
(http://aswhite.net/media/papers/white_universal_2016.pdf). 
To appear in *Proceedings of the Conference on Empirical Methods in Natural Language Processing 2016*.
'''

# parse the WSD dataset and construct X_Y tensors
def parse_data():

    # parse the EUD-EWT conllu files and retrieve the sentences
    # remove all punctuation?
    train_file = open("data/UD_English-EWT/en_ewt-ud-train.conllu", "r", encoding="utf-8")
    train_data = list(parse_incr(train_file))
    # train_data = [[''.join(c for c in word.get('lemma') if c not in string.punctuation) for word in token_list] for token_list in train_data]
    # train_data = [[word for word in s if word] for s in train_data]
    print('Parsed {} training data from UD_English-EWT/en_ewt-ud-train.conllu.'.format(len(train_data)))

    test_file = open("data/UD_English-EWT/en_ewt-ud-test.conllu", "r", encoding="utf-8")
    test_data = list(parse_incr(test_file))
    # test_data = [[''.join(c for c in word.get('lemma') if c not in string.punctuation) for word in token_list] for token_list in test_data]
    # test_data = [[word for word in s if word] for s in test_data]
    print('Parsed {} testing data from UD_English-EWT/en_ewt-ud-test.conllu.'.format(len(test_data)))

    dev_file = open("data/UD_English-EWT/en_ewt-ud-dev.conllu", "r", encoding="utf-8")
    dev_data = list(parse_incr(dev_file))
    # dev_data = [[''.join(c for c in word.get('lemma') if c not in string.punctuation) for word in token_list] for token_list in dev_data]
    # dev_data = [[word for word in s if word] for s in dev_data]
    print('Parsed {} dev data from UD_English-EWT/en_ewt-ud-dev.conllu.'.format(len(dev_data)))

    # parse the WSD dataset
    wsd_data = []

    # read in tsv by White et. al., 2016
    with open('data/wsd/wsd_eng_ud1.2_10262016.tsv', mode = 'r') as wsd_file:

        tsv_reader = csv.DictReader(wsd_file, delimiter = '\t')      

        # store the data: ordered dict row
        for row in tsv_reader:                                

            # each data vector
            wsd_data.append(row)

        # make sure all data are parsed
        print('Parsed {} word sense data from White et. al., 2016.'.format(len(wsd_data)))

    return wsd_data, train_data, test_data, dev_data

In [5]:
# parse the data
wsd_data, train_data, test_data, dev_data = parse_data()

# return the raw sentences from the EUD for train, test, and dev
# test the first 20 sentences
all_senses, all_definitions = get_all_senses_and_definitions(wsd_data)


Parsed 12543 training data from UD_English-EWT/en_ewt-ud-train.conllu.
Parsed 2077 testing data from UD_English-EWT/en_ewt-ud-test.conllu.
Parsed 2002 dev data from UD_English-EWT/en_ewt-ud-dev.conllu.
Parsed 439312 word sense data from White et. al., 2016.


In [6]:
'''
Construct the X and Y for train, dev, and test from White et. al., 2016
For each anonator and each word, on pair of data and label will be created
Warning: code here is hard to read LMAO
'''
def construct_X_Y(all_senses, all_definitions, train_data, dev_data, test_data):
    
    wsd_data = []
    
    with open('data/wsd/wsd_eng_ud1.2_10262016.tsv', mode = 'r') as wsd_file:
        
        tsv_reader = csv.DictReader(wsd_file, delimiter = '\t')
        
        # same annotator and same sentence number will generate on pair of X_Y
        # manually set the first sentence from White et. al., 2016
        current_annotator = '0'
        current_sentence_num = '1364'
        current_Y = [0 for _ in range(len(all_senses['spring']))]           
        sentence = train_data[1363]
        
        # word from the EUD is a ordered dict for word properties
        # use key 'lemma' to get the literal representations
        current_X = [word.get('lemma') for word in sentence]
        current_idx = 12
        
        # lists X and Y
        train_X, test_X, dev_X = ([] for i in range(3))
        train_Y, test_Y, dev_Y = ([] for i in range(3))
        train_word_idx, test_word_idx, dev_word_idx = ([] for i in range(3))

        for idx, row in enumerate(tsv_reader):
                        
            # training set; only test first 30 training sentences for now
            if idx < 60 and row['Split'] == 'train':
                
                # if still is the same annotatior, word index, target word
                # modify Y with the sense reponse
                if current_annotator == row['Annotator.ID'] and current_idx == int(row['Arg.Token']) - 1 and current_sentence_num == row['Sentence.ID'].split(' ')[-1]:
                    
                    sense_idx = all_senses[row['Arg.Lemma']].index(row['Synset'])
                    if row['Sense.Response'] == 'True':
                        current_Y[sense_idx] = 1
                    else:
                        current_Y[sense_idx] = 0
                
                # if switch annotator or target word
                # append the Y and X from the last annotator and word
                # start a new Y and X for the current annotator and target
                else:
                    # print('h2: {}'.format(idx))
                    # sentence
                    train_X.append(current_X)
                    # annotator responses, e.g., [1, 0, ...]
                    train_Y.append(current_Y)
                    train_word_idx.append(current_idx)
                    
                    current_annotator = row['Annotator.ID']
                    current_sentence_num = row['Sentence.ID'].split(' ')[-1]
                    
                    current_idx = int(row['Arg.Token']) - 1
                    current_Y = [0 for _ in range(len(all_senses[row['Arg.Lemma']]))]
                    sense_idx = all_senses[row['Arg.Lemma']].index(row['Synset'])
                    if row['Sense.Response'] == 'True':
                        current_Y[sense_idx] = 1
                    else:
                        current_Y[sense_idx] = 0
                    
                    sentence_id = row['Sentence.ID']
                    sentence_number = int(sentence_id.split(' ')[-1]) - 1
                    sentence = train_data[sentence_number]
                    current_X = [word.get('lemma') for word in sentence]
                    
            # testing set
            elif idx < 60 and row['Split'] == 'test':
                
                if current_annotator == row['Annotator.ID'] and current_idx == int(row['Arg.Token']) - 1 and current_sentence_num == row['Sentence.ID'].split(' ')[-1]:
                    
                    sense_idx = all_senses[row['Arg.Lemma']].index(row['Synset'])
                    if row['Sense.Response'] == 'True':
                        current_Y[sense_idx] = 1
                    else:
                        current_Y[sense_idx] = 0
                else:
                    test_X.append(current_X)
                    test_Y.append(current_Y)
                    test_word_idx.append(current_idx)

                    current_annotator = row['Annotator.ID']
                    current_sentence_num = row['Sentence.ID'].split(' ')[-1]

                    current_idx = int(row['Arg.Token']) - 1
                    current_Y = [0 for _ in range(len(all_senses[row['Arg.Lemma']]))]
                    sense_idx = all_senses[row['Arg.Lemma']].index(row['Synset'])
                    if row['Sense.Response'] == 'True':
                        current_Y[sense_idx] = 1
                    else:
                        current_Y[sense_idx] = 0
                    
                    sentence_id = row['Sentence.ID']
                    sentence_number = int(sentence_id.split(' ')[-1]) - 1
                    sentence = test_data[sentence_number]
                    current_X = [word.get('lemma') for word in sentence]
                    
            # dev set       
            elif idx < 60:
                if current_annotator == row['Annotator.ID'] and current_idx == int(row['Arg.Token']) - 1 and current_sentence_num == row['Sentence.ID'].split(' ')[-1]:
                    
                    sense_idx = all_senses[row['Arg.Lemma']].index(row['Synset'])
                    if row['Sense.Response'] == 'True':
                        current_Y[sense_idx] = 1
                    else:
                        current_Y[sense_idx] = 0
                else:
                    dev_X.append(current_X)
                    dev_Y.append(current_Y)
                    dev_word_idx.append(current_idx)

                    current_annotator = row['Annotator.ID']
                    current_sentence_num = row['Sentence.ID'].split(' ')[-1]

                    current_idx = int(row['Arg.Token']) - 1
                    current_Y = [0 for _ in range(len(all_senses[row['Arg.Lemma']]))]
                    sense_idx = all_senses[row['Arg.Lemma']].index(row['Synset'])
                    if row['Sense.Response'] == 'True':
                        current_Y[sense_idx] = 1
                    else:
                        current_Y[sense_idx] = 0
                    
                    sentence_id = row['Sentence.ID']
                    sentence_number = int(sentence_id.split(' ')[-1]) - 1
                    sentence = dev_data[sentence_number]
                    current_X = [word.get('lemma') for word in sentence]
        
        print('\n******************* Data Example ***********************')
        print('Sentence: {}'.format(train_X[0]))
        print('Annotator Response, i.e., true label: {}'.format(train_Y[0]))
        print('Target Word Index: {}'.format(train_word_idx[0]))
        print('All senses for the target word: {}'.format(all_senses[train_X[0][train_word_idx[0]]]))
        print('All definitions (in order of its senses from WordNet): {}'.format(all_definitions[train_X[0][train_word_idx[0]]]))
        print('********************************************************')
        
        return train_X, test_X, dev_X, train_Y, test_Y, dev_Y, train_word_idx, test_word_idx, dev_word_idx
                    

In [7]:
# get the X and Y for models
train_X, test_X, dev_X, train_Y, test_Y, dev_Y, train_word_idx, test_word_idx, dev_word_idx = construct_X_Y(all_senses, all_definitions, train_data, dev_data, test_data)



******************* Data Example ***********************
Sentence: ['on', 'August', '9', ',', '2004', ',', 'it', 'be', 'announce', 'that', 'in', 'the', 'spring', 'of', '2001', ',', 'a', 'man', 'name', 'El', '-', 'Shukrijumah', ',', 'also', 'know', 'as', 'Jafar', 'the', 'Pilot', ',', 'who', 'be', 'part', 'of', 'a', '"', 'second', 'wave', ',', '"', 'have', 'be', 'case', 'New', 'York', 'City', 'helicopter', '.']
Annotator Response, i.e., true label: [1, 0, 0, 0, 0, 0]
Target Word Index: 12
All senses for the target word: ['spring.n.01', 'spring.n.02', 'spring.n.03', 'spring.n.04', 'give.n.01', 'leap.n.01']
All definitions (in order of its senses from WordNet): [['the', 'season', 'of', 'growth'], ['a', 'metal', 'elastic', 'device', 'that', 'returns', 'to', 'its', 'shape', 'or', 'position', 'when', 'pushed', 'or', 'pulled', 'or', 'pressed'], ['a', 'natural', 'flow', 'of', 'ground', 'water'], ['a', 'point', 'at', 'which', 'water', 'issues', 'forth'], ['the', 'elasticity', 'of', 'something',

In [8]:
from model import *
from trainer import *
'''
from torchviz import make_dot

model = Model(all_senses = all_senses, elmo_class = elmo)
for name, param in model.named_parameters():
    if param.requires_grad == True:
        print(name, param.size())
        
output = model(train_X[0], train_word_idx[0])
print(output.size())

make_dot(output, model.parameters())
'''

Device: cpu


'\nfrom torchviz import make_dot\n\nmodel = Model(all_senses = all_senses, elmo_class = elmo)\nfor name, param in model.named_parameters():\n    if param.requires_grad == True:\n        print(name, param.size())\n        \noutput = model(train_X[0], train_word_idx[0])\nprint(output.size())\n\nmake_dot(output, model.parameters())\n'

In [9]:
epochs = 50
trainer = Trainer(epochs = epochs, elmo_class = elmo, all_senses = all_senses)

In [11]:
train_losses, dev_losses, dev_rs = trainer.train(train_X, train_Y, train_word_idx, dev_X, dev_Y, dev_word_idx, development = False)



#############   Model Parameters   ##############
layers.word_sense.0.weight torch.Size([512, 512])
layers.word_sense.0.bias torch.Size([512])
layers.word_sense.2.weight torch.Size([300, 512])
layers.word_sense.2.bias torch.Size([300])
dimension_reduction_MLP.weight torch.Size([256, 3072])
dimension_reduction_MLP.bias torch.Size([256])
wsd_lstm.weight_ih_l0 torch.Size([1024, 256])
wsd_lstm.weight_hh_l0 torch.Size([1024, 256])
wsd_lstm.bias_ih_l0 torch.Size([1024])
wsd_lstm.bias_hh_l0 torch.Size([1024])
wsd_lstm.weight_ih_l0_reverse torch.Size([1024, 256])
wsd_lstm.weight_hh_l0_reverse torch.Size([1024, 256])
wsd_lstm.bias_ih_l0_reverse torch.Size([1024])
wsd_lstm.bias_hh_l0_reverse torch.Size([1024])
wsd_lstm.weight_ih_l1 torch.Size([1024, 512])
wsd_lstm.weight_hh_l1 torch.Size([1024, 256])
wsd_lstm.bias_ih_l1 torch.Size([1024])
wsd_lstm.bias_hh_l1 torch.Size([1024])
wsd_lstm.weight_ih_l1_reverse torch.Size([1024, 512])
wsd_lstm.weight_hh_l1_reverse torch.Size([1024, 256])
wsd_lstm.bia


Epoch: 1, Mean Training Loss: 2.9486106634140015
[Epoch: 2/50]



Epoch: 2, Mean Training Loss: 2.9463888804117837
[Epoch: 3/50]



Epoch: 3, Mean Training Loss: 2.9290804167588553
[Epoch: 4/50]



Epoch: 4, Mean Training Loss: 2.919999192158381
[Epoch: 5/50]


...


Epoch: 46, Mean Training Loss: 2.9080555737018585
[Epoch: 47/50]



Epoch: 47, Mean Training Loss: 2.9091667532920837
[Epoch: 48/50]



Epoch: 48, Mean Training Loss: 2.910555640856425
[Epoch: 49/50]



Epoch: 49, Mean Training Loss: 2.9122223059336343
[Epoch: 50/50]



Epoch: 50, Mean Training Loss: 2.9113889733950296
tensor([[0, 0, 0, 0, 0, 0],
        [0, 0, 0, 0, 0, 0],
        [0, 0, 0, 0, 0, 0],
        ...,
        [0, 0, 0, 0, 0, 0],
        [0, 0, 0, 0, 0, 0],
        [0, 1, 0, 0, 0, 1]], dtype=torch.uint8)


In [12]:
# plot the learning curve
import matplotlib
import matplotlib.pyplot as plt
ite = [e for e in range(epochs)]
plt.plot(train_losses, label = "Cosine Similarity Loss")
plt.legend(loc = "best")
title = "Learning Curve (# of training examples " + str(len(train_X)) + ")"
plt.title(title)
plt.ylabel('Loss')
plt.xlabel('Number of Iteration')
plt.savefig('loss.png')
